In [1]:
# import dependencies
from sklearn.preprocessing import LabelEncoder
from sqlalchemy import create_engine
from config import dbConnectionString
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
engine = create_engine(dbConnectionString)

#import dataframe from SQL database
df = pd.read_sql_query('select * from "cardioData"',con=engine)
df = df.drop(columns = ['index', 'id'])
df

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active
0,50.4,Female,168.0,62.0,False,23,110,80,normal,normal,False,False,True
1,55.4,Male,156.0,85.0,True,46,140,90,well above normal,normal,False,False,True
2,51.7,Male,165.0,64.0,True,25,130,70,well above normal,normal,False,False,False
3,47.9,Male,156.0,56.0,False,20,100,60,normal,normal,False,False,False
4,60.0,Male,151.0,67.0,False,30,120,80,above normal,above normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34,120,80,normal,normal,True,False,True
63912,61.9,Male,158.0,126.0,True,100,140,90,above normal,above normal,False,False,True
63913,52.2,Female,183.0,105.0,True,60,180,90,well above normal,normal,False,True,False
63914,61.5,Male,163.0,72.0,True,32,135,80,normal,well above normal,False,False,False


In [3]:
le = LabelEncoder()
df2 = df.copy()
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active
0,50.4,Female,168.0,62.0,False,23,110,80,normal,normal,False,False,True
1,55.4,Male,156.0,85.0,True,46,140,90,well above normal,normal,False,False,True
2,51.7,Male,165.0,64.0,True,25,130,70,well above normal,normal,False,False,False
3,47.9,Male,156.0,56.0,False,20,100,60,normal,normal,False,False,False
4,60.0,Male,151.0,67.0,False,30,120,80,above normal,above normal,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34,120,80,normal,normal,True,False,True
63912,61.9,Male,158.0,126.0,True,100,140,90,above normal,above normal,False,False,True
63913,52.2,Female,183.0,105.0,True,60,180,90,well above normal,normal,False,True,False
63914,61.5,Male,163.0,72.0,True,32,135,80,normal,well above normal,False,False,False


In [4]:
df2['BMI'] = df2['BMI'].astype(float)

In [5]:
df2['BMI_class'] = ''

for i in range(len(df2['age'])):
    if df2['BMI'][i] < 18.5:
        df2['BMI_class'][i] = 'underweight'
    elif df2['BMI'][i] < 24.9 and df2['BMI'][i] > 18.5:
        df2['BMI_class'][i] = 'normal'
    elif df2['BMI'][i] < 29.9 and df2['BMI'][i] > 24.9:
        df2['BMI_class'][i] = 'overweight'
    elif df2['BMI'][i] < 34.9 and df2['BMI'][i] > 29.9:
        df2['BMI_class'][i] = 'obese'
    elif df2['BMI'][i] < 39.9 and df2['BMI'][i] > 34.9:
        df2['BMI_class'][i] = 'obese2'
    else:
        df2['BMI_class'][i] = 'obese3'
        
df2['ap_hi_class'] = ''

for i in range(len(df2['age'])):
    if df2['ap_hi'][i] < 120:
        df2['ap_hi_class'][i] = 'normal'
    elif df2['ap_hi'][i] < 129 and df2['ap_hi'][i] >= 120:
        df2['ap_hi_class'][i] = 'elevated'
    elif df2['ap_hi'][i] < 139 and df2['ap_hi'][i] >= 130:
        df2['ap_hi_class'][i] = 'highbloodpressure'
    elif df2['ap_hi'][i] < 180 and df2['ap_hi'][i] >= 140:
        df2['ap_hi_class'][i] = 'highbloodpressure2'
    else:
        df2['ap_hi_class'][i] = 'hypertensive'
        
df2['ap_lo_class'] = ''

for i in range(len(df2['age'])):
    if df2['ap_lo'][i] < 80:
        df2['ap_lo_class'][i] = 'elevated'
    elif df2['ap_lo'][i] < 80 and df2['ap_lo'][i] >= 89:
        df2['ap_lo_class'][i] = 'highbloodpressure'
    elif df2['ap_lo'][i] < 120 and df2['ap_lo'][i] >= 90:
        df2['ap_lo_class'][i] = 'highbloodpressure2'
    else:
        df2['ap_lo_class'][i] = 'hypertensive'
        
df2

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,Female,168.0,62.0,False,23.0,110,80,normal,normal,False,False,True,normal,normal,hypertensive
1,55.4,Male,156.0,85.0,True,46.0,140,90,well above normal,normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
2,51.7,Male,165.0,64.0,True,25.0,130,70,well above normal,normal,False,False,False,overweight,highbloodpressure,elevated
3,47.9,Male,156.0,56.0,False,20.0,100,60,normal,normal,False,False,False,normal,normal,elevated
4,60.0,Male,151.0,67.0,False,30.0,120,80,above normal,above normal,False,False,False,obese,elevated,hypertensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,Female,168.0,76.0,False,34.0,120,80,normal,normal,True,False,True,obese,elevated,hypertensive
63912,61.9,Male,158.0,126.0,True,100.0,140,90,above normal,above normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
63913,52.2,Female,183.0,105.0,True,60.0,180,90,well above normal,normal,False,True,False,obese3,hypertensive,highbloodpressure2
63914,61.5,Male,163.0,72.0,True,32.0,135,80,normal,well above normal,False,False,False,obese,highbloodpressure,hypertensive


In [6]:
#female = 0, male = 1
df2['gender'] = le.fit_transform(df2['gender'])
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,0,168.0,62.0,False,23.0,110,80,normal,normal,False,False,True,normal,normal,hypertensive
1,55.4,1,156.0,85.0,True,46.0,140,90,well above normal,normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
2,51.7,1,165.0,64.0,True,25.0,130,70,well above normal,normal,False,False,False,overweight,highbloodpressure,elevated
3,47.9,1,156.0,56.0,False,20.0,100,60,normal,normal,False,False,False,normal,normal,elevated
4,60.0,1,151.0,67.0,False,30.0,120,80,above normal,above normal,False,False,False,obese,elevated,hypertensive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,0,168.0,76.0,False,34.0,120,80,normal,normal,True,False,True,obese,elevated,hypertensive
63912,61.9,1,158.0,126.0,True,100.0,140,90,above normal,above normal,False,False,True,obese3,highbloodpressure2,highbloodpressure2
63913,52.2,0,183.0,105.0,True,60.0,180,90,well above normal,normal,False,True,False,obese3,hypertensive,highbloodpressure2
63914,61.5,1,163.0,72.0,True,32.0,135,80,normal,well above normal,False,False,False,obese,highbloodpressure,hypertensive


In [7]:
normal_dic = {
    "normal" : 0,
    "above normal" : 1,
    "well above normal" : 2,
}

true_false_dic = {
    True : 1,
    False : 0,
}

BMI_dic = {
    "underweight" : 0,
    "normal" : 1,
    "overweight" : 2, 
    "obese": 3, 
    "obese2" : 4, 
    "obese3" : 5
}

blood_pressure_dic = {
    "normal" : 0,
    "elevated" : 1, 
    "highbloodpressure": 2, 
    "highbloodpressure2": 3,
    "hypertensive" : 4
}

df2['cardiovascular'] = df2['cardiovascular'].apply(lambda x : true_false_dic[x])
df2['smoke'] = df2['smoke'].apply(lambda x : true_false_dic[x])
df2['alchohol'] = df2['alchohol'].apply(lambda x : true_false_dic[x])
df2['active'] = df2['active'].apply(lambda x : true_false_dic[x])

df2['cholestrol'] = df2['cholestrol'].apply(lambda x : normal_dic[x])
df2['gloucose'] = df2['gloucose'].apply(lambda x : normal_dic[x])
df2['BMI_class'] = df2['BMI_class'].apply(lambda x : BMI_dic[x])
df2['ap_hi_class'] = df2['ap_hi_class'].apply(lambda x : blood_pressure_dic[x])
df2['ap_lo_class'] = df2['ap_lo_class'].apply(lambda x : blood_pressure_dic[x])
df2

,age,gender,height,weight,cardiovascular,BMI,ap_hi,ap_lo,cholestrol,gloucose,smoke,alchohol,active,BMI_class,ap_hi_class,ap_lo_class
0,50.4,0,168.0,62.0,0,23.0,110,80,0,0,0,0,1,1,0,4
1,55.4,1,156.0,85.0,1,46.0,140,90,2,0,0,0,1,5,3,3
2,51.7,1,165.0,64.0,1,25.0,130,70,2,0,0,0,0,2,2,1
3,47.9,1,156.0,56.0,0,20.0,100,60,0,0,0,0,0,1,0,1
4,60.0,1,151.0,67.0,0,30.0,120,80,1,1,0,0,0,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,52.7,0,168.0,76.0,0,34.0,120,80,0,0,1,0,1,3,1,4
63912,61.9,1,158.0,126.0,1,100.0,140,90,1,1,0,0,1,5,3,3
63913,52.2,0,183.0,105.0,1,60.0,180,90,2,0,0,1,0,5,4,3
63914,61.5,1,163.0,72.0,1,32.0,135,80,0,2,0,0,0,3,2,4


In [14]:
# set X and y values
X = df2.drop(['cardiovascular', 'BMI', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular'].values

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [25]:
y

0        0
1        1
2        1
3        0
4        0
        ..
63911    0
63912    1
63913    1
63914    1
63915    0
Name: cardiovascular, Length: 63916, dtype: int64

In [16]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
nn_model = tf.keras.models.Sequential()

nn = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="ReLU"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 13)                156       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 112       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 349
Trainable params: 349
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=67)

Epoch 1/67
1499/1499 [==============================] - 1s 362us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 2/67
1499/1499 [==============================] - 1s 370us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 3/67
1499/1499 [==============================] - 1s 350us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 4/67
1499/1499 [==============================] - 1s 357us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 5/67
1499/1499 [==============================] - 1s 356us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 6/67
1499/1499 [==============================] - 1s 368us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 7/67
1499/1499 [==============================] - 1s 383us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 8/67
1499/1499 [==============================] - 1s 379us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 9/67
1499/1499 [==============================] - 1s 379us/step - loss: 4.7490 - accuracy: 0.1124
Epoch 10/67
1499/1499 [==============================] - 1s 385u

KeyboardInterrupt: 

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

500/500 - 0s - loss: 4.7500 - accuracy: 0.1097
Loss: 4.7499847412109375, Accuracy: 0.10970649123191833


## Drop a few features

In [ ]:
df2

In [21]:
X = df2.drop(['cardiovascular', 'BMI', 'gender', 'cholestrol', 'gloucose', 'smoke', 'alchohol', 'active'], 1).values
y = df2['cardiovascular']

C:\Users\Fadl\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [22]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_1 = 80
hidden_nodes_2 = 80
hidden_nodes_3 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_1, input_dim=input_features, activation="ReLU"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_2, activation="ReLU"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_3, activation="ReLU"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
X = df2.drop(['cardiovascular', 'BMI'], 1).values
y = df2['cardiovascular'].values

In [ ]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 13
hidden_nodes_layer2 = 8
nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()


In [ ]:
# compile model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=110)

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
X = df2.drop(['cardiovascular', 'BMI', 'gender', 'cholestrol', 'gloucose', 'smoke', 'alchohol', 'active'], 1).values
y = df2['cardiovascular'].values

In [ ]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()


In [ ]:
# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 13
hidden_nodes_layer2 = 8
nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()


In [ ]:
# compile model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=110)

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
X = df2.drop(['cardiovascular', 'BMI', 'gender', 'cholestrol', 'gloucose', 'smoke', 'alchohol', 'active'], 1).values
y = df2['cardiovascular'].values

In [ ]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9, stratify=y)

scaler= StandardScaler()


In [ ]:
# neural net
input_features = len(X_train_scaled[0])


nn_model = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=13, input_dim=input_features, activation="sigmoid"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="sigmoid"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()


In [ ]:
# compile model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# compile model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# train model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=40)

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# compile model
nn_model.compile(loss="binary_crossentropy", optimizer="Adagrad", metrics=["accuracy"])

# train model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=110)

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# compile model
nn_model.compile(loss="binary_crossentropy", optimizer="Adamax", metrics=["accuracy"])

# train model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=110)

model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

## Try Bucketing - Use BMI first

In [ ]:
X = df2.drop(['cardiovascular', 'height', 'weight', 'BMI_class', 'ap_hi_class', 'ap_lo_class'], 1).values
y = df2['cardiovascular'].values

# split data into training and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

scaler= StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# neural net
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  13
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 8
hidden_nodes_layer5 = 3
nn_model = tf.keras.models.Sequential()

nn = tf.keras.models.Sequential()

# first layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="ReLU"))

# second layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="ReLU"))

# third layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="ReLU"))

# Fourth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="ReLU"))

# Fifth layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="ReLU"))

# output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# check the structure
nn_model.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=67)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")